<a href="https://colab.research.google.com/github/trizakiilu/Capstone-2023/blob/main/Copy_of_Attendance_System_Rev03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face_recognition
!pip install click
!pip install schedule
!pip install timeloop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=4dd4535487810ff06c5eab2dbc03458d666d7ae976be0c0a260fafe5337094c9
  Stored in directory: /root/.cache/pip/wheels/22/a8/60/4a2aeb763d63f50190f4c4e07069a22245347eeafdb3a67551
Successfully built face-recognition-models
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for timeloop: filename=timeloop-1.0.2-py3-none-any.whl size=3717 sha256=be883b5a903

In [ ]:
from google.colab import drive
drive.mount('/content/drive/' , force_remount=True)
import face_recognition
import numpy as np
import cv2
import pandas as pd
import os
from google.colab.patches import cv2_imshow
import os
import PIL
import io
from base64 import b64decode, b64encode
import time
from datetime import datetime
import xlrd
import pandas as pd
import time
from datetime import date
import csv
today = date.today().strftime('%Y-%m-%d %H:%M:%S')
os.chdir('/content/drive/MyDrive/attendance')
import Video_Capture
capture_number = 0
import click
import schedule
import time
import datetime
import pytz



Mounted at /content/drive/


In [ ]:
# Set the timezone to your desired timezone
tz = pytz.timezone('America/New_York')

e = datetime.datetime.now(tz)
print(e)
start_hour = 12
strat_minute = 25
finish_hour = 12
finish_minute = 26
time_to_capture = 60  # waiting time for capture next photo (second)

2023-03-28 12:32:32.787275-04:00


In [ ]:
def streaming_video(count):
    a = Video_Capture
    capture_number = 0
    video_stream = a.Video_Capture(1).video_stream
    video_frame = a.Video_Capture(1).video_frame
    js_to_image = a.Video_Capture(1).js_to_image
    bbox_to_bytes = a.Video_Capture(1).bbox_to_bytes

    video_stream()
    face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    

    while True:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break
        # convert JS response to OpenCV Image
        image_bytes = b64decode(js_reply["img"].split(',')[1])         
        #img = js_to_image(js_reply["img"])
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        # decode numpy array into OpenCV BGR image
        img = cv2.imdecode(jpg_as_np, flags=1)
        # create transparent overlay for bounding box
        bbox_array = np.zeros([600,600,4], dtype=np.uint8)

        # grayscale image for face detection
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #face_location = face_recognition.face_locations(img)
        # get face region coordinates
        face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") 
        faces = face_cascade.detectMultiScale(gray)
        # get face bounding box for overlay
        if len(faces) > 0:
          for (x,y,w,h) in faces:
            bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w+10,y+h+10),(255,0,0),2)
          os.chdir('/content/drive/MyDrive/attendance/test/')
          filename = f"{count+1}.png"
          bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
          # convert overlay of bbox into bytes
          bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
          #iobuf = io.BytesIO()
          # format bbox into png for return
          bbox_PIL.save(filename)
          # format return string
          #bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
            # update bbox so next frame gets new overlay
          #bbox = bbox_bytes
          ### inja ##########################
          cv2.imwrite(filename, img)
          capture_number += 1
          #a.Video_Capture(0).release()
          cv2.destroyAllWindows()
          
          break
    return True

In [ ]:
count = 0
while True:
  if e.hour >= start_hour and e.hour <= finish_hour:
    if e.minute >= strat_minute and e.minute <= finish_minute:
      print('Its time for rollcall')
      streaming_video(count)
      print('Face Detected')
      time.sleep(time_to_capture)
      e = datetime.datetime.now(tz)
      count =+ 1
    else:
      e = datetime.datetime.now(tz)
      print("The houre is out of rollcall time ")
      break
  else:
    e = datetime.datetime.now(tz)
    print("The houre is out of rollcall time ")
    break
  


The houre is out of rollcall time 


In [ ]:
def read_train_images(folder , dim):
  X = []
  encode = []
  y = []
  Y = []
  n_samples=0
  none_image = 0
  for i in os.listdir(folder):
      #print(i) 
      img_file = cv2.imread( folder +'/' + i)
      #cv2_imshow(img_file)
      if img_file is not None:
          img_orginal=cv2.resize(img_file,dim)
          face_location = face_recognition.face_locations(img_orginal)
          encoded_img = face_recognition.face_encodings(img_orginal, face_location)
          #print(face_location)            
          X.append(img_orginal)
          encode.append(encoded_img)
          name = os.path.split(i)[-1]
          Y.append(name)
          n_samples+=1    
      else:
        none_image=+1
        print(f'there is {none_image} "not image" file in this folder')

  return X , Y, encode
dim = (600,600)
train_folder =  "/content/drive/MyDrive/attendance/train"
x , y , encoded_images= read_train_images(train_folder, dim)

In [ ]:
def read_test_image(test_folder, dim, encoded_images, y):
  Attendees_list = []
  k=0
  for iter in os.listdir(test_folder):
    test_img = cv2.imread(test_folder + iter)
    j = 0
    for i in encoded_images:
      test_img=cv2.resize(test_img,dim)
      face_location_test = face_recognition.face_locations(test_img)
      encoded_test = face_recognition.face_encodings(test_img, face_location_test)
      #print(len(encoded_test))
      if len(encoded_test) == 0:
        results = []
        
      else:
        results = face_recognition.compare_faces(np.array(i), np.array(encoded_test))
        print(results)
        if len(results) == 0:
          print('No face were detected')
        else:
          if results[0]:
            Attendees_list.append(y[j])
          else:
            pass
      j+=1
    k+=1   
  return Attendees_list, k
test_folder = "/content/drive/MyDrive/attendance/test/"
raw_list, K = read_test_image(test_folder, dim, encoded_images, y)


[True]
[False]
[]
No face were detected
[False]
[]
No face were detected
[False]
[]
No face were detected
[]
No face were detected


In [ ]:
lis_of_attendees = []
[lis_of_attendees.append(x) for x in raw_list if x not in lis_of_attendees]

absent_list = list(set(y).difference(lis_of_attendees))
Absents = []
for i in absent_list:
  name = i.split('.')[0]
  Absents.append(name)

Attendees = []
for i in lis_of_attendees:
  name = i.split('.')[0]
  Attendees.append(name)
print(f'Attendees students are :{Attendees}')
print(f'Absents students are :{Absents}')

if K>len(raw_list):
  strange_no = K-len(raw_list)
 # print(f'There are {strange_no} strange person in the class. Be careful!!')
  os.chdir('/content/drive/MyDrive/attendance/')


Attendees students are :['Amin']
Absents students are :['Sherea', 'brad', 'jack', 'Yesha', 'Triza', 'sara', 'John']


In [ ]:
#making report
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment,Border,Side
from openpyxl import Workbook, load_workbook


#make time and date dynamic

# Define the data as lists
class_name = "A"
time = "5:15 to 5:20"
date = "2023-03-27"

numbers = [1, 2, 3, 4, 5, 6, 7, 8]
#names = ["amin", "yesha", "triza", "sherea", "jack", "john", "sara", "soozan"]
student_ids = [605, 608, 604, 606, 609, 602, 603, 601]
#statuses = ["absent", "absent", "absent", "absent", "absent", "absent", "absent", "absent"]
descriptions = ["", "", "", "", "", "", "", ""]

# Combine attendance and absent lists
names = lis_of_attendees + absent_list

 #Create a list of statuses
statuses = []
for person in names:
    if person in lis_of_attendees:
        statuses.append("present")
    else:
        statuses.append("absent")

# Open the Excel file
#workbook = openpyxl.Workbook()

# Select the worksheet you want to modify
#worksheet = workbook.active

In [ ]:
workbook =  load_workbook(filename="classlist.xlsx")

worksheet = workbook.active

# Merge cells and make text bold
bold_font = Font(bold=True)
worksheet.merge_cells('A1:E1')
worksheet['A1'].font = bold_font
worksheet.merge_cells('A2:E2')
worksheet['A2'].font = bold_font

# Set border style of bottom edge to None for merged cells A1:E1 and A2:E2
for row in worksheet.iter_rows(min_row=1, max_row=2, min_col=1, max_col=5):
    for cell in row:
        border = Border(bottom=Side(border_style=None))
        cell.border = border
        
# Write the data to specific cells
worksheet['A1'] = 'Class ' + class_name
worksheet['F1'] = 'time: '
worksheet['G1'] =  time
worksheet['F2'] = 'date: '
worksheet['G2'] =  date

worksheet['A3'] = 'number'
worksheet['B3'] = 'name'
worksheet['C3'] = 'student id'
worksheet['D3'] = 'status'
worksheet['E3'] = 'description'



# Define fill colors for status field
present_fill = PatternFill(fill_type='solid', start_color='CEE4CE', end_color='CEE4CE')
absent_fill = PatternFill(fill_type='solid', start_color='FACACA', end_color='FACACA')

for i in range(len(numbers)):
    row = i + 6
    worksheet.cell(row=row, column=1, value=numbers[i])
    worksheet.cell(row=row, column=2, value=names[i])
    worksheet.cell(row=row, column=3, value=student_ids[i])
    worksheet.cell(row=row, column=4, value=statuses[i])
    worksheet.cell(row=row, column=5, value=descriptions[i])
    
    # Set the fill color based on the value of the "status" field
    if statuses[i] == 'present':
        worksheet.cell(row=row, column=4).fill = present_fill
    else:
        worksheet.cell(row=row, column=4).fill = absent_fill

# Save the changes to the Excel file
workbook.save('classlist.xlsx')

In [ ]:
#mailing report to teacher
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE
from email import encoders
from google.colab import drive



# Set email parameters
sender = 'durhamattendance@gmail.com'
password = 'ytsfrxfvssiaucms'
recipient = 'aminysf23@gmail.com'
subject = 'Test Email with Excel attachment'
body = 'Please find attached the Excel file.'

# Set file parameters
filename = 'classlist.xlsx'
filepath = '/content/drive/MyDrive/attendance/' + filename

# Create email message
msg = MIMEMultipart()
msg['From'] = sender
msg['To'] = recipient
msg['Subject'] = subject
msg.attach(MIMEText(body))

# Attach file to email message
with open(filepath, "rb") as attachment:
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename= {filename}")
    msg.attach(part)

# Send email using SMTP server
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender, password)
    server.sendmail(sender, [recipient], msg.as_string())
    server.quit()
    print(f'Success: Email sent to {recipient} with attachment {filename}.')
except Exception as e:
    print(f'Error: Unable to send email to {recipient}.')
    print(e)


Success: Email sent to aminysf23@gmail.com with attachment classlist.xlsx.


In [ ]:
from flask import Flask, render_template, request
import google.auth
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import pandas as pd

app = Flask(__name__)

# Define the credentials required to access your Google Drive account
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json')

In [ ]:

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Set the scopes
SCOPES = ['https://www.googleapis.com/auth/drive']

# Set the path to your credentials file
creds = Credentials.from_authorized_user_file('path/to/credentials.json', SCOPES)

# If the credentials don't exist, run the authentication flow
if not creds:
    flow = InstalledAppFlow.from_client_secrets_file('path/to/client_secret.json', SCOPES)
    creds = flow.run_local_server(port=0)

# Build the Drive API client
drive_service = build('drive', 'v3', credentials=creds)

FileNotFoundError: ignored